In [ ]:
# pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import uuid
import requests
import os
import re
import json

In [ ]:
with open('_input.txt', 'r', encoding='utf') as f:
  html = f.read()

html = BeautifulSoup(html, 'lxml')
html = html.find('div', class_='flexlayout__tab')

problem_name = html.find('div').find('div').find('div').find('div').find('div').find("a").get_text()
problem_name = problem_name.split('.')[1:]
problem_name = ".".join(problem_name).strip()
problem = html.find('div', class_='elfjS')
images = problem.find_all('img')
img = []
if len(images) > 0:
  if os.path.exists("assets"):
    files = os.listdir("assets")
    for file in files:
      os.remove(os.path.join("assets", file))
    os.removedirs("assets")
  os.makedirs("assets", exist_ok=True)
for image in images:
  src = image.get('src')
  file_extension = src.split(".")[-1]
  file_name = uuid.uuid4().__str__().replace("-", "")
  response = requests.get(src)
  if response.status_code == 200:
    with open(f"assets/{file_name}.{file_extension}", "wb") as f:
      f.write(response.content)
  else:
    response = requests.get(src)
    if response.status_code == 200:
      with open(f"{file_name}.{file_extension}", "wb") as f:
        f.write(response.content)
  img.append({
    "source": src,
    "target": f"assets/{file_name}.{file_extension}"
  })

template = f"""
<h3>{problem_name}</h3>
{problem}
"""

template = template \
  .replace(" class=\"elfjS\" data-track-load=\"description_content\"", "") \
  .replace(" class=\"example\"", "")

for image in img:
  template = template.replace(image['source'], image['target'])

f = open("README.md", "w", encoding='utf-8')
f.write(template)
f.close()

In [ ]:
with open('_input.txt', 'r', encoding='utf') as f:
  html = f.read()
notebook = {
  "cells": [],
  "metadata": {
    "kernelspec": {
      "display_name": "base",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.11.7"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 2
}

html = BeautifulSoup(html, 'lxml')

first_line = html.find('div', class_='view-lines monaco-mouse-cursor-text') \
  .find_all('div')[1].get_text().replace('\xa0', ' ').replace('  ', ' ')
start = first_line.index('def ') + 4
end = first_line.index('(')
function_name = first_line[start:end]

pattern = r'\((.*?)\)'
match = re.search(pattern, first_line)
args = match.group(1)
args = args.split(',')[1:]
arguments = []
for arg in args:
  name, arg_type = arg.strip().split(': ')
  arguments.append((name, arg_type))

import_list = 'List' in first_line
import_optional = 'Optional' in first_line
need_import = []
if import_list:
  need_import.append('from typing import List\n')
if import_optional:
  need_import.append('from typing import Optional\n')

first_cell = {
  "cell_type": "code",
  "metadata": {},
  "outputs": [],
  "source": [
    *need_import,
    "class Solution:\n",
    first_line + "\n",
    "    pass"
  ]
}

notebook["cells"].append(first_cell)

example = html.find('div', class_='flexlayout__tab').find('div', class_='elfjS').get_text()
pattern = r"Input:\s*(.*?)\n"
inputs = re.findall(pattern, example)

for input in inputs:
  args = ', '.join([f'{name}={name}' for name, _ in arguments])
  input = input.split(', ')
  for i in range(len(input)):
    input[i] = input[i].replace(',', ', ') + '\n'

  cell = {
    "cell_type": "code",
    "metadata": {},
    "outputs": [],
    "source": [
      *input,
      f"Solution().{function_name}({args})"
    ]
  }

  notebook["cells"].append(cell)

with open('main.ipynb', 'w', encoding='utf') as f:
  f.write(json.dumps(notebook, indent=2))